# Продвинутое машинное обучение: ДЗ 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/andrewponomarev/made_ml_2/blob/main/homeworks/hw3/MCMC.ipynb#scrollTo=0qRrSzwQgemx)

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена.

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода.


In [ ]:
import os
import string
from collections import Counter
import numpy as np
import random
from copy import copy
from tqdm import tqdm


import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
random_state = 42

## Содержание

0. [Загрузка и обработка данных](#data_load)
1. [Частотное декодирование на основе униграмм](#unigramm)
2. [Частотное декодирование на основе биграмм](#bigramm)
3. [Декодирование с помощью MCMC алгоритма](#mcmc)
4. [Расшифровка тестового текста](#decoding)
5. [Сравнение моделей](#n_gramm_mcmc)
6. [Применение модели](#model_usage)

## Загрузка и обработка данных <a name = "data_load"/>

In [ ]:
DATA_PATH = 'data'

with open(os.path.join(DATA_PATH, 'AnnaKarenina.txt'), 'r') as f1, \
     open(os.path.join(DATA_PATH, 'WarAndPeace.txt'), 'r') as f2:
    anna_karenina = f1.read().lower()
    war_and_peace = f2.read().lower()

In [ ]:
train_text = anna_karenina + war_and_peace

## 1. Частотное декодирование на основе униграмм <a name = "unigramm"/>

Реализуйте базовый частотный метод по Шерлоку Холмсу:

* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

Зададим два алфавита: английский и русский

In [ ]:
ENG_ALPHABET = string.ascii_lowercase + ' '
RU_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

In [ ]:
class FrequencyDecoder: 
    
    def __init__(self, language: str = "ru", n_gram_len: int = 1):
        self.language = language
        self.n_gram_len = n_gram_len
        if (self.language == "ru"):
            self.__alphabet = RU_ALPHABET
        elif (self.language == "eng"):
            self.__alphabet = ENG_ALPHABET
        else:
            raise ValueError("Wrong language")
            
    def train(self, text: str):
        text = self.preprocess(text)
        n_grams = self.__get_ngrams(text)
        self.__frequency_dict = self.__get_frequency_dict(n_grams)
        
    def decode(self, text: str) -> str:
        text += ' ' * (len(text) % self.n_gram_len)
        n_grams = self.__get_ngrams(text)
        decoded_frequency_dict = self.__get_frequency_dict(n_grams)
        interim_size = min(len(decoded_frequency_dict), len(self.__frequency_dict))
        interim = {}
        for i in range(interim_size):
            interim[[*decoded_frequency_dict][i]] = [*self.__frequency_dict][i]
        return ''.join([interim[ng] for ng in n_grams])[:len(text)]
        
        
    def preprocess(self, text: str):
        return ''.join([c for c in text.lower() if c in self.__alphabet])
    
    def __get_ngrams(self, text: str) -> list:
        return [text[i:i+self.n_gram_len] for i in range(len(text) - self.n_gram_len + 1)]
    
    def __get_frequency_dict(self, n_grams: list) -> dict:
        return {k: v for k, v in sorted(Counter(n_grams).items(), key=lambda item: -item[1])}
        
    

In [ ]:
class ReplaceEncoder:
    
    def __init__(self, language: str = "ru"):
        self.language = language
        if (self.language == "ru"):
            self.__alphabet = RU_ALPHABET
        elif (self.language == "eng"):
            self.__alphabet = ENG_ALPHABET
        else:
            raise ValueError("Wrong language")
            
        self.__permutation = ''.join(random.sample(self.__alphabet,len(self.__alphabet)))
        self.__translator = str.maketrans(self.__alphabet, self.__permutation)
            
            
    def encode(self, text: str) -> str:
        return self.__translate(self.__preprocess(text))
    
    
    def __preprocess(self, text: str) -> str:
        return ''.join([c for c in text.lower() if c in self.__alphabet])
    
    
    def __translate(self, text: str) -> str:
        return text.translate(self.__translator)
    

Подсчитаем частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить

In [ ]:
unigram_decoder = FrequencyDecoder('ru')
unigram_decoder.train(train_text)

Возьмем тестовый тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), 

In [ ]:
test_text = """
Проснувшись, мы пересмотрели все добро, награбленное шайкой на разбитом пароходе; \
там оказались и сапоги, и одеяла, и платья, и всякие другие вещи, а еще много книг, \
подзорная труба и три ящика сигар. Такими богачами мы с Джимом еще никогда в жизни не были. \
Сигары оказались первый сорт. До вечера мы валялись в лесу и разговаривали; я читал книжки; \
и вообще мы недурно провели время. Я рассказал Джиму обо всем, что произошло на пароходе и на пароме, \
и сообщил ему кстати, что это и называется приключением; а он ответил, \
что не желает больше никаких приключений. Джим рассказал, что в ту минуту, \
когда я залез в рубку, а он прокрался обратно к плоту и увидел, что плота больше нет, \
он чуть не умер со страха: так и решил, что ему теперь крышка, чем бы дело ни кончилось, \
потому что если его не спасут, так он утонет; а если кто нибудь его спасет, так отвезет домой, \
чтобы получить за него награду, а там мисс Уотсон, наверно, продаст его на Юг. \
Что ж, он был прав; он почти всегда бывал прав, голова у него работала здорово, - для негра, конечно.
"""

In [ ]:
print("Текст в изначальном виде:\n")
print(test_text)

Текст в изначальном виде:


Проснувшись, мы пересмотрели все добро, награбленное шайкой на разбитом пароходе; там оказались и сапоги, и одеяла, и платья, и всякие другие вещи, а еще много книг, подзорная труба и три ящика сигар. Такими богачами мы с Джимом еще никогда в жизни не были. Сигары оказались первый сорт. До вечера мы валялись в лесу и разговаривали; я читал книжки; и вообще мы недурно провели время. Я рассказал Джиму обо всем, что произошло на пароходе и на пароме, и сообщил ему кстати, что это и называется приключением; а он ответил, что не желает больше никаких приключений. Джим рассказал, что в ту минуту, когда я залез в рубку, а он прокрался обратно к плоту и увидел, что плота больше нет, он чуть не умер со страха: так и решил, что ему теперь крышка, чем бы дело ни кончилось, потому что если его не спасут, так он утонет; а если кто нибудь его спасет, так отвезет домой, чтобы получить за него награду, а там мисс Уотсон, наверно, продаст его на Юг. Что ж, он был прав; он по

Зашифруем их посредством случайной перестановки символов;

In [ ]:
replace_encoder = ReplaceEncoder('ru')
encoded_test_text = replace_encoder.encode(test_text)
print("Зашифрованный случайными перестановками текст:\n")
print(encoded_test_text)

Зашифрованный случайными перестановками текст:

йы цштуирцчавбайъыъцв лыъмрауцъаь ёы ашеэыеёмъшш ъаиеос оашеаыепёрл вайеы н ьъалева сепемрцчарацей эрара ьъзмеараймелчзарауцзсръаьытэръауъжраеаъжъавш э асшрэай ьп ышезалытёеаралыразжрсеацрэеыалесрвраё эефевравбацаьюрв ваъжъашрс эьеауаюрпшрашъаёбмрацрэеыба сепемрцчайъыубоац ылаь ауъфъыеавбауемзмрцчауамъцтараыепэ уеыруемразафрлемасшрюсрарау  ёжъавбашъьтыш айы уъмрауыъвзазаыеццсепемаьюрвта ё ауцъвафл айы рп им ашеайеы н ьъарашеайеы въарац  ёжрмаъвтасцлелрафл акл арашепбуеълцзайырсмдфъшръваеа ша луълрмафл ашъаюъмеълаё мчиъашрсесрнайырсмдфъшроаьюрваыеццсепемафл ауалтаврштлтас эьеазапемъпауаытёстаеа шайы сыемцза ёыелш асайм лтаратурьъмафл айм леаё мчиъашъла шафтлчашъатвъыац ацлыенеалесараыъирмафл аъвталъйъычасыбисеафъваёбаьъм ашрас шфрм цчай л втафл аъцмраъэ ашъацйецтлалеса шатл шълаеаъцмрасл ашрётьчаъэ ацйецълалеса луъпълаь в оафл ёбай мтфрлчапеашъэ ашеэыеьтаеалеваврццат лц шашеуъыш айы ьецлаъэ ашеадэафл аюа шаёбмайыеуа шай флрауцъэьеаёбуемайы

Расшифруем их частотным методом.

In [ ]:
decoded_test_text = unigram_decoder.decode(encoded_test_text)
print("Расшифрованный текст с помощью частотной униграммы:\n")
print(decoded_test_text)

Расшифрованный текст с помощью частотной униграммы:

мсовтукжнвч дб масавдоисалн ква погсо теясеглаттоа жехрох те сеыгниод месоюопа иед ореыелнвч н вемоян н опазле н млеичз н квзрна псуяна кашн е аша дтояо ртня мопыостез исуге н исн зшнре вняес иерндн гояеьедн дб в пйндод аша тнрояпе к йнытн та гблн вняесб ореыелнвч маскбх воси по каьасе дб келзлнвч к лаву н сеыяокеснкелн з ьниел ртнйрн н коогша дб тапусто мсокалн ксадз з севвреыел пйнду ого квад ьио мсоныожло те месоюопа н те месода н воогшнл аду рвиеин ьио цио н теыбкеаивз мснрлэьатнад е от оикаинл ьио та йалеаи голчжа тнрерню мснрлэьатнх пйнд севвреыел ьио к иу днтуиу рояпе з ыелаы к сугру е от мсорселвз огсеито р млоиу н укнпал ьио млоие голчжа таи от ьуич та удас во висеюе иер н сажнл ьио аду иамасч рсбжре ьад гб пало тн ротьнловч моиоду ьио авлн аяо та вмевуи иер от уиотаи е авлн рио тнгупч аяо вмеваи иер оикаыаи подох ьиогб молуьнич ые таяо теясепу е иед днвв уоивот текасто мсопеви аяо те эя ьио й от гбл мсек от моьин кваяпе гбк

Введем метрику качества декодирования, как долю верно расшифрованных символов:

In [ ]:
def accuracy(orig_text, decoded_text):
    return sum(np.array(list(orig_text)) == np.array(list(decoded_text))) / len(orig_text)

In [ ]:
print("Частотный алгоритм на униграмммах")
print(f"Доля верно расшифрованных символовов {accuracy(unigram_decoder.preprocess(test_text), decoded_test_text)}")

Частотный алгоритм на униграмммах
Доля верно расшифрованных символовов 0.3396414342629482


## 2. Частотное декодирование на основе биграмм <a name = "bigramm"/>

В результате не получилась хорошая расшифровка. Так что давайте сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.

In [ ]:
bigram_decoder = FrequencyDecoder('ru', 2)
bigram_decoder.train(train_text)

In [ ]:
decoded_test_text_2 = bigram_decoder.decode(encoded_test_text)
print("Расшифрованный текст с помощью частотной биграммы:\n")
print(decoded_test_text_2)

Расшифрованный текст с помощью частотной биграммы:

коневыкуылар хейзан онм одро соболвеаксивсй ы ветакао товида новазтьеднеи е  тсою  виеих лмоятелжа нжиийтувнскоечте  та  б вотрыыйче птеен сомернеис ж еле нал койен оаву оторнаказан оня о нощечианк дио я о  о елеепше за я о  с а зсянылсния о товисьош ус  новмнннымдис  нтов мемио ога л мех  нм улелк  ди   холаимев сан еебча эки тлуниалы ннпеька я о алы быо  рньмириу а ноадимиверняал кил ужеемо прдек ивутпаойемо м одронобоовэтсежевстеенл мех  не ларискк му га то ммасе яьнлао е ст нпрт бекао ноадимиверееро оаву оторнаказан он соболажезопчтноин эбрвоовази тов знесол ва м одротолинаожшеказан онто мус лаксвь я о  б вотум дтилиербывшлинакао  рни итрче кнара  холакнгд уо я о тоиздутьлях  нм одрое стувияугкиели  сконетив такао тоз вевашани рни б ваттвичу оторнараовэтсежеить  отьдеи товидаваен иго пи  сконетн ячабуывреи е  та  сомернеис ж еле ня о е  та  сомернетеса ня о ноиндутьлямидорал ваить   лыая ксяамо  иго пи жнд  пи я о е  тотаесплиудос

In [ ]:
print("Частотный алгоритм на биграммах")
print(f"Доля верно расшифрованных символовов {accuracy(bigram_decoder.preprocess(test_text), decoded_test_text_2)}")

Частотный алгоритм на биграммах
Доля верно расшифрованных символовов 0.06772908366533864


Стало еще хуже, потому что количество биграмм намного больше, чем количество букв, попасть сложнее. 

## 3. Декодирование с помощью MCMC алгоритма <a name = "mcmc"/>

Основная часть задания — в том, как можно улучшить биграммы.

* Предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* Реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Алгоритм следующий. Повторяем n_iter раз:

* Меняем местами две случайные буквы в "перестановке шифрования"
* Оцениваем вероятность получить деловдирванный с помощью этой перестановки текст. У нас есть наша "языковая модель", а точнее частотный словарь n-грамм, построенный на тренировочных текстах. Из него мы знаем вероятности встретить n-граммы. Таким образом, мы можем посчитать likelihood декодированного текста как произведение вероятностей для всех n-грамм в нем.
* Принимаем или отклоняем перестановку шифрования (если новый likelihood больше, то принимаем, если меньше, то принимаем с вероятностью new_llh / old_llh)

In [ ]:
class MCMCDecoder:
    
    def __init__(self, n_gram_len, language = "ru", n_iter=20000, verbose=5000):
        
        self.language = language
        self.n_gram_len = n_gram_len
        if (self.language == "ru"):
            self.__alphabet = RU_ALPHABET
        elif (self.language == "eng"):
            self.__alphabet = ENG_ALPHABET
        else:
            raise ValueError("Wrong language")
        
        self.n_iter = n_iter
        self.verbose = verbose
        
        
    def loglikelihood(self, text):
        text += ' ' * (len(text) % self.n_gram_len)
        n_grams = self.__get_ngrams(text)
        frequency_dict = self.__get_frequency_dict(n_grams)
        return np.sum([count * np.log(self.__frequency_dict.get(ngram, 1 / len(self.__alphabet) ** 2)) 
                       for ngram, count in frequency_dict.items()])
    
    
    def __get_ngrams(self, text: str) -> list:
        return [text[i:i+self.n_gram_len] for i in range(len(text) - self.n_gram_len + 1)]
    
     
    def __get_frequency_dict(self, n_grams: list) -> dict:
        return {k: v for k, v in sorted(Counter(n_grams).items(), key=lambda item: -item[1])}
             
    def __mutate(self, text):
        tl = list(text)
        letters = np.random.choice(list(self.__alphabet), 2, replace=False)
        for i in range(len(text)):
            if tl[i] == letters[0]:
                tl[i] = letters[1]
            elif tl[i] == letters[1]:
                tl[i] = letters[0]
        return ''.join(tl)
    
    def __accept(self, cur_llh, new_llh):
        if new_llh > cur_llh:
            return True
        return np.random.rand() < np.exp(new_llh - cur_llh)
    
    def preprocess(self, text: str):
        return ''.join([c for c in text.lower() if c in self.__alphabet])
    
    def train(self, text):
        text = self.preprocess(text)
        n_grams = self.__get_ngrams(text)
        self.__frequency_dict = self.__get_frequency_dict(n_grams)
        
        
    def decode(self, text, n_iter=20000, is_verbose = True):
        best_decoded_text = copy(text)
        cur_llh = best_llh = self.loglikelihood(text) 
        for iteration in tqdm(range(n_iter)):
            new_text = self.__mutate(copy(text))
            new_llh = self.loglikelihood(new_text)
            if self.__accept(cur_llh, new_llh):
                text = new_text
                cur_llh = new_llh
                if cur_llh > best_llh:
                    best_llh = cur_llh
                    best_decoded_text = copy(text)
                    
            if is_verbose and iteration % self.verbose == 0:
                print(''.join(best_decoded_text))
                print('--------------------------------------------------------------------------------')
                
        return ''.join(best_decoded_text)

In [ ]:
mcmc_decoder = MCMCDecoder(2, 'ru')
mcmc_decoder.train(train_text)

In [ ]:
decoded_test_text_3 = mcmc_decoder.decode(encoded_test_text)
print("MCMC алгоритм на биграммах\n")
print(decoded_test_text_3)
print("\n")
print(f"Доля верно расшифрованных символовов {accuracy(mcmc_decoder.preprocess(test_text), decoded_test_text_3)}")

  1%|          | 105/20000 [00:00<00:58, 342.51it/s]

йы гштуиргчавбайъыъгв лыъмраугъаь ёы ашеэыеёмъшш ъаиеос оашеаыепёрл вайеы н ьъалева сепемргчарагей эрара ьъзмеараймелчзараугзсръаьытэръауъжраеаъжъавш э асшрэай ьп ышезалытёеаралыразжрсеагрэеыалесрвраё эефевравбагаьюрв ваъжъашрс эьеауаюрпшрашъаёбмрагрэеыба сепемргчайъыубоаг ылаь ауъфъыеавбауемзмргчауамъгтараыепэ уеыруемразафрлемасшрюсрарау  ёжъавбашъьтыш айы уъмрауыъвзазаыеггсепемаьюрвта ё аугъвафл айы рп им ашеайеы н ьъарашеайеы въараг  ёжрмаъвтасглелрафл акл арашепбуеългзайырсмдфъшръваеа ша луълрмафл ашъаюъмеълаё мчиъашрсесрнайырсмдфъшроаьюрваыеггсепемафл ауалтаврштлтас эьеазапемъпауаытёстаеа шайы сыемгза ёыелш асайм лтаратурьъмафл айм леаё мчиъашъла шафтлчашъатвъыаг аглыенеалесараыъирмафл аъвталъйъычасыбисеафъваёбаьъм ашрас шфрм гчай л втафл аъгмраъэ ашъагйегтлалеса шатл шълаеаъгмрасл ашрётьчаъэ агйегълалеса луъпълаь в оафл ёбай мтфрлчапеашъэ ашеэыеьтаеалеваврггат лг шашеуъыш айы ьеглаъэ ашеадэафл аюа шаёбмайыеуа шай флраугъэьеаёбуемайыеуаэ м уеаташъэ аыеё лемеапь ы у ааьмзашъэыеас ш

 26%|██▌       | 5155/20000 [00:13<00:37, 397.49it/s]

проснувшись мы пересмотрели все добро награбленное шайкой на разбитом пароходе там оказались и сапоги и одеяла и платья и всякие другие вещи а еще много книг подзорная труба и три ящика сигар такими богачами мы с джимом еще никогда в жизни не были сигары оказались первый сорт до вечера мы валялись в лесу и разговаривали я читал книжки и вообще мы недурно провели время я рассказал джиму обо всем что произошло на пароходе и на пароме и сообщил ему кстати что это и называется приключением а он ответил что не желает больше никаких приключений джим рассказал что в ту минуту когда я залез в рубку а он прокрался обратно к плоту и увидел что плота больше нет он чуть не умер со страха так и решил что ему теперь крышка чем бы дело ни кончилось потому что если его не спасут так он утонет а если кто нибудь его спасет так отвезет домой чтобы получить за него награду а там мисс уотсон наверно продаст его на юг что ж он был прав он почти всегда бывал прав голова у него работала здорово  для негра кон

 50%|█████     | 10083/20000 [00:26<00:23, 417.89it/s]

проснувшись мы пересмотрели все добро награбленное шайкой на разбитом пароходе там оказались и сапоги и одеяла и платья и всякие другие вещи а еще много книг подзорная труба и три ящика сигар такими богачами мы с джимом еще никогда в жизни не были сигары оказались первый сорт до вечера мы валялись в лесу и разговаривали я читал книжки и вообще мы недурно провели время я рассказал джиму обо всем что произошло на пароходе и на пароме и сообщил ему кстати что это и называется приключением а он ответил что не желает больше никаких приключений джим рассказал что в ту минуту когда я залез в рубку а он прокрался обратно к плоту и увидел что плота больше нет он чуть не умер со страха так и решил что ему теперь крышка чем бы дело ни кончилось потому что если его не спасут так он утонет а если кто нибудь его спасет так отвезет домой чтобы получить за него награду а там мисс уотсон наверно продаст его на юг что ж он был прав он почти всегда бывал прав голова у него работала здорово  для негра кон

 75%|███████▌  | 15048/20000 [00:39<00:13, 359.36it/s]

проснувшись мы пересмотрели все добро награбленное шайкой на разбитом пароходе там оказались и сапоги и одеяла и платья и всякие другие вещи а еще много книг подзорная труба и три ящика сигар такими богачами мы с джимом еще никогда в жизни не были сигары оказались первый сорт до вечера мы валялись в лесу и разговаривали я читал книжки и вообще мы недурно провели время я рассказал джиму обо всем что произошло на пароходе и на пароме и сообщил ему кстати что это и называется приключением а он ответил что не желает больше никаких приключений джим рассказал что в ту минуту когда я залез в рубку а он прокрался обратно к плоту и увидел что плота больше нет он чуть не умер со страха так и решил что ему теперь крышка чем бы дело ни кончилось потому что если его не спасут так он утонет а если кто нибудь его спасет так отвезет домой чтобы получить за него награду а там мисс уотсон наверно продаст его на юг что ж он был прав он почти всегда бывал прав голова у него работала здорово  для негра кон

100%|██████████| 20000/20000 [00:53<00:00, 372.43it/s]


MCMC алгоритм на биграммах

проснувшись мы пересмотрели все добро награбленное шайкой на разбитом пароходе там оказались и сапоги и одеяла и платья и всякие другие вещи а еще много книг подзорная труба и три ящика сигар такими богачами мы с джимом еще никогда в жизни не были сигары оказались первый сорт до вечера мы валялись в лесу и разговаривали я читал книжки и вообще мы недурно провели время я рассказал джиму обо всем что произошло на пароходе и на пароме и сообщил ему кстати что это и называется приключением а он ответил что не желает больше никаких приключений джим рассказал что в ту минуту когда я залез в рубку а он прокрался обратно к плоту и увидел что плота больше нет он чуть не умер со страха так и решил что ему теперь крышка чем бы дело ни кончилось потому что если его не спасут так он утонет а если кто нибудь его спасет так отвезет домой чтобы получить за него награду а там мисс уотсон наверно продаст его на юг что ж он был прав он почти всегда бывал прав голова у него раб

## 4. Расшифровка тестового текста <a name ="decoding"/>

Расшифруем еще одно закодированное сообщение

In [ ]:
secret_message = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

Переведем в алфавит, на котором обучались

In [ ]:
encoded_letters = ''.join(set(secret_message))
rus_letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '[:len(encoded_letters)]
secret_message_ru = secret_message.translate(str.maketrans(encoded_letters, rus_letters))

In [ ]:
decoded_secret_message = mcmc_decoder.decode(secret_message_ru, n_iter=60000)
print(decoded_secret_message)

  0%|          | 204/60000 [00:00<01:10, 853.48it/s] 

уъдйещрещйлйауесбхптдчсржейдйеобнайесбхптдчсржеаугъаезеёабцбеъббшфусйиегбабхржедуцгбеохбнйатачеъгбхууещъуцбещрещъуеълудтдйеохтщйдчсбейеобдзнйауептгъйптдчсржештддемтеобъдулсууенуащухабуемтлтсйуегзхътекбаиегбсунсбеиесйнуцбесуебшуфтв
--------------------------------------------------------------------------------


  9%|▊         | 5125/60000 [00:04<00:46, 1185.99it/s]

если бы бимите новзальный или дорти новзальный текст у этого соожчения котовый легко дворитать сковее бсего бы бсе смелали двабильно и долурите заксизальный жалл па дослемнее ретбевтое памание кувса хотя конерно я нирего не ожечаш
--------------------------------------------------------------------------------


 17%|█▋        | 10312/60000 [00:10<00:53, 924.89it/s]

...

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
--------------------------------------------------------------------------------


 92%|█████████▏| 55247/60000 [00:53<00:04, 1156.39it/s]

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
--------------------------------------------------------------------------------


100%|██████████| 60000/60000 [00:58<00:00, 1033.88it/s]

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


## 5. Сравнение моделей  <a name="n_gramm_mcmc"/>

In [ ]:
def print_quality(train_text, test_text, encoded_text_text, lang = 'ru'):
    
    for n_gram in range(1,10):
        mcmc_decoder = MCMCDecoder(n_gram, lang)
        mcmc_decoder.train(train_text)
        decoded_test_text = mcmc_decoder.decode(encoded_text_text,  is_verbose = False)
        acc = accuracy(mcmc_decoder.preprocess(test_text), decoded_test_text)
        print(f"Доля верно расшифрованных символовов {acc} для {n_gram}-грамм")

In [ ]:
print_quality(train_text, test_text, encoded_test_text)

100%|██████████| 20000/20000 [00:36<00:00, 548.42it/s]


Доля верно расшифрованных символовов 0.3655378486055777 для 1-грамм


100%|██████████| 20000/20000 [01:12<00:00, 275.81it/s]


Доля верно расшифрованных символовов 0.08366533864541832 для 2-грамм


100%|██████████| 20000/20000 [01:39<00:00, 200.01it/s]


...

Доля верно расшифрованных символовов 0.12151394422310757 для 7-грамм


100%|██████████| 20000/20000 [02:04<00:00, 161.19it/s]


Доля верно расшифрованных символовов 0.20916334661354583 для 8-грамм


100%|██████████| 20000/20000 [01:45<00:00, 189.82it/s]

Доля верно расшифрованных символовов 0.0 для 9-грамм


Как видим, если n > 3, то алгоритм уже не может давать хорошее качество. Логично предположить, что если увеличить обучающий корпус текстов, то можно повыить качество в том числе и на больших n.

## 6. Применение модели  <a name="model_usage"/>

Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Данную модель можно использовать например для восстановления исходного текста в ситуации когда сбивается кодировка, при условии, что каждому символу в неправильной кодировке соответствует один символ в правильной.